In [1]:
import pandas as pd
data = pd.read_csv("C:/Users/Daniel/Documents/Masterarbeit/mup_valid_articles.csv", encoding="utf-8")

In [2]:
# load the functionalities
from entitynetwork.enititynetwork_pipeline.process_article import *
from entitynetwork.database_creation.fill_database import *
from entitynetwork.relation_extraction.relation_extraction import *
#from entitynetwork.validation_tool.relation_validation import *

In [3]:
# Establishe DB Connection
g = connect_database()


In [4]:
# Tag selection

selected_tag = ["B-PERSON", "I-PERSON", "B-FAC", "I-FAC", "B-ORG", "I-ORG", "B-GPE", "I-GPE",
                "B-LOC", "I-LOC", "B-WORK_OF_ART", "I-WORK_OF_ART", "B-LAW", "I-LAW"]

In [5]:
# Start the validation and store the data
for i in range(len(data)):
    print("Current Article" + str(i))
    article = Article(data["text"][i], data["author_extracted"][i], data["date"][i], data["source"][i], data["id"][i],
               download_model=False, selected_tags=selected_tag, n_shift=1,graph=g)
    article.process_articles()

Current Article0
Current Article1
Current Article2
Current Article3
Current Article4
Current Article5
Current Article6
Current Article7
Current Article8
Current Article9
Current Article10
Current Article11
Current Article12
Current Article13
Current Article14
Current Article15
Current Article16
Current Article17
Current Article18
Current Article19
Current Article20
Current Article21
Current Article22
Current Article23
Current Article24
Current Article25
Current Article26
Current Article27
Current Article28
Current Article29
Current Article30
Current Article31
Current Article32
Current Article33
Current Article34
Current Article35
Current Article36
Current Article37
Current Article38
Current Article39
Current Article40
Current Article41
Current Article42
Current Article43
Current Article44
Current Article45
Current Article46
Current Article47
Current Article48
Current Article49
Current Article50
Current Article51
Current Article52
Current Article53
Current Article54
Current Article55
Cu

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2020-07-06 02:39:05.722 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from C:\Users\Daniel\.deeppavlov\models\ner_ontonotes_bert_mult\tag.dict]
2020-07-06 02:39:45.966 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from C:\Users\Daniel\.deeppavlov\models\

In [8]:
cy_1 = "CALL gds.alpha.degree.stream({nodeQuery: 'MATCH (a:Article) WHERE a.id = 1 WITH a.id as ids "
cy_2 = "MATCH (e:Entity)-[r:OCCURS_WITH]-() WHERE r.article IN ids RETURN id(e) AS id',relationshipQuery: "
cy_3 = "'MATCH (a:Article) WHERE a.id = 1 WITH a.id as ids MATCH (e:Entity)-[r:OCCURS_WITH]-(b:Entity) "
cy_4 = "WHERE r.article IN ids RETURN id(e) AS source, id(b) AS target'}) YIELD nodeId, score WITH "
cy_5 = "gds.util.asNode(nodeId).name AS user, score MATCH (e:Entity)-[r:OCCURS_IN]->(a:Article) WHERE e.name = user "
cy_6 = "AND a.id = 1 AND score > 0 SET r.degree = toInteger(score)"
old_id = "1"
for i in range(len(data)):
    cy_1 = cy_1.replace(old_id, str(data["id"][i]))
    cy_3 = cy_3.replace(old_id, str(data["id"][i]))
    cy_6 = cy_6.replace(old_id, str(data["id"][i]))
    old_id = str(data["id"][i])
    if g.evaluate("MATCH p=()-[r:OCCURS_WITH]->() WHERE r.article = $id RETURN count(p)", parameters={"id": int(data["id"][i])})>0:
        print(old_id)
        g.run(cy_1+cy_2+cy_3+cy_4+cy_5+cy_6)

350
351
352
354
355
357
358
359
360
361
362
363
364
365
366
367
370
371
372
373
375
377
379
380
382
383
384
385
386
391
392
393
394
397
399
402
403
404
405
406
407
408
409
410
411
412
413
414
415
418
419
420
421
422
423
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
442
443
444
445
446
447
448
449
453
454
455
459
460
461
462
464
465
467
468
470
471
474
477
478
479
481
482
483
485
486
491
503
504
505
508
509
510
511
512
513
515
516
517
518
519
520
521
523
525
526
530
531
534
536
537
538
540
541
542
544
545
547
548
551
552
553
554
555
556
557
558
559
560
562
564
565
566
567
569
571
572
573
574
575
576
577
578
579
580
581
582
584
585
586
587
588
589
591
592
593
594
595
596
597
598
600
601
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
627
628
629
630
632
633
634
635
636
637
638
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
660
661
662
663
664
666
667
668
669
671
672
674
675
678
679
680
681
682
683
684
685


In [9]:
cy_1 = "CALL gds.alpha.betweenness.stream({nodeQuery: 'MATCH (a:Article) WHERE a.id = 1 WITH a.id as ids "
cy_2 = "MATCH (e:Entity)-[r:OCCURS_WITH]-() WHERE r.article IN ids RETURN id(e) AS id',relationshipQuery: "
cy_3 = "'MATCH (a:Article) WHERE a.id = 1 WITH a.id as ids MATCH (e:Entity)-[r:OCCURS_WITH]-(b:Entity) "
cy_4 = "WHERE r.article IN ids RETURN id(e) AS source, id(b) AS target'}) YIELD nodeId, centrality WITH "
cy_5 = "gds.util.asNode(nodeId).name AS user, centrality MATCH (e:Entity)-[r:OCCURS_IN]->(a:Article) WHERE e.name = user "
cy_6 = "AND a.id = 1 AND centrality > 0 SET r.centrality = centrality"
old_id = "1"
for i in range(len(data)):
    cy_1 = cy_1.replace(old_id, str(data["id"][i]))
    cy_3 = cy_3.replace(old_id, str(data["id"][i]))
    cy_6 = cy_6.replace(old_id, str(data["id"][i]))
    old_id = str(data["id"][i])
    if g.evaluate("MATCH p=()-[r:OCCURS_WITH]->() WHERE r.article = $id RETURN count(p)", parameters={"id": int(data["id"][i])})>0:
        print(old_id)
        g.run(cy_1+cy_2+cy_3+cy_4+cy_5+cy_6)


350
351
352
354
355
357
358
359
360
361
362
363
364
365
366
367
370
371
372
373
375
377
379
380
382
383
384
385
386
391
392
393
394
397
399
402
403
404
405
406
407
408
409
410
411
412
413
414
415
418
419
420
421
422
423
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
442
443
444
445
446
447
448
449
453
454
455
459
460
461
462
464
465
467
468
470
471
474
477
478
479
481
482
483
485
486
491
503
504
505
508
509
510
511
512
513
515
516
517
518
519
520
521
523
525
526
530
531
534
536
537
538
540
541
542
544
545
547
548
551
552
553
554
555
556
557
558
559
560
562
564
565
566
567
569
571
572
573
574
575
576
577
578
579
580
581
582
584
585
586
587
588
589
591
592
593
594
595
596
597
598
600
601
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
627
628
629
630
632
633
634
635
636
637
638
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
660
661
662
663
664
666
667
668
669
671
672
674
675
678
679
680
681
682
683
684
685
